<a href="https://colab.research.google.com/github/Chandanachelliboyina/Chandana_Fmml_Course_assignment/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [3]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [4]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [6]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [7]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

3 ans)there's no one-size-fits-all answer to what percentage of the dataset should be reserved for the validation set, as the ideal split depends on several factors, including the size of your dataset, the complexity of your model, and the nature of your problem. However, there are some common guidelines and practices that can help you make an informed decision:

80/20 or 70/30 Split: A common starting point is to reserve 20% or 30% of your data for the validation set and use the remaining 80% or 70% for training. This is a reasonable choice for medium-sized datasets.

Cross-Validation: If your dataset is relatively small, you might consider using k-fold cross-validation, where the data is divided into k subsets, and you perform training and validation k times, rotating the validation set each time. This can provide a more robust estimate of model performance, especially when you have limited data.

Large Datasets: For very large datasets, you can afford to allocate a smaller percentage to the validation set since you still have a substantial amount of data for training. In such cases, a 90/10 or even 95/5 split might be appropriate.

Imbalanced Data: If your dataset is imbalanced (one class significantly outnumbers the others), you might want to ensure that the validation set retains the same class distribution as the training set. Techniques like stratified sampling can help with this.

Experimentation: Ultimately, the best percentage to reserve for the validation set may require some experimentation. You can try different splits and see how they affect your model's performance on a held-out test set. The goal is to find a balance that results in a model that generalizes well to unseen data.

Consider Data Availability: In some cases, you may have limited data, and reserving a large portion for the validation set may not be feasible. In such situations, you may need to make trade-offs and consider techniques like data augmentation, transfer learning, or leveraging pre-trained models to compensate for the limited data.

Remember that the validation set's primary purpose is to provide an estimate of your model's performance and help you make decisions about hyperparameters and model

2 ans)The accuracy of a machine learning model on the validation set can be affected by the percentage of data allocated to the validation set. The validation set plays a crucial role in evaluating the model's performance and generalization ability. Here's how changing the percentage of the validation set can impact your model:

1. **Increased Validation Set Percentage**:
   - **Pros**:
     - More data for validation: A larger validation set provides a more representative sample of the data, which can lead to a more reliable estimate of the model's performance.
     - Robustness: With a larger validation set, the model's performance evaluation is less likely to be affected by random variations in the data.
   - **Cons**:
     - Less training data: By allocating a larger percentage to the validation set, you reduce the amount of data available for training the model. This can lead to longer training times and potentially underfitting if the training data is insufficient.

2. **Reduced Validation Set Percentage**:
   - **Pros**:
     - More training data: With a smaller validation set, you have more data available for training the model, which can help the model learn better and potentially achieve better performance.
     - Faster training: Smaller validation sets lead to shorter training times since the model is trained on a larger portion of the data.
   - **Cons**:
     - Unreliable evaluation: A smaller validation set may not provide an accurate representation of the model's true performance. It can be sensitive to noise and outliers, leading to overfitting if the model performs well on this specific subset but poorly on unseen data.
     - Risk of overfitting: With less validation data, it becomes harder to detect overfitting, where the model performs well on the validation set but poorly on unseen data.

The choice of the percentage allocated to the validation set depends on several factors, including the size of your overall dataset, the complexity of your model, and the available computational resources. It's often a trade-off between having a reliable estimate of performance (larger validation set) and ensuring the model is adequately trained (larger training set).

Typically, a common split is 70-80% for training and 20-30% for validation. However, this can vary depending on the specific problem and dataset. You may also consider techniques like cross-validation, which involve splitting the data into multiple subsets for validation to get a more robust estimate of model performance.

The accuracy of a machine learning model on the validation set can be affected by the percentage of data allocated to the validation set. The validation set plays a crucial role in evaluating the model's performance and generalization ability. Here's how changing the percentage of the validation set can impact your model:

1. **Increased Validation Set Percentage**:
   - **Pros**:
     - More data for validation: A larger validation set provides a more representative sample of the data, which can lead to a more reliable estimate of the model's performance.
     - Robustness: With a larger validation set, the model's performance evaluation is less likely to be affected by random variations in the data.
   - **Cons**:
     - Less training data: By allocating a larger percentage to the validation set, you reduce the amount of data available for training the model. This can lead to longer training times and potentially underfitting if the training data is insufficient.

2. **Reduced Validation Set Percentage**:
   - **Pros**:
     - More training data: With a smaller validation set, you have more data available for training the model, which can help the model learn better and potentially achieve better performance.
     - Faster training: Smaller validation sets lead to shorter training times since the model is trained on a larger portion of the data.
   - **Cons**:
     - Unreliable evaluation: A smaller validation set may not provide an accurate representation of the model's true performance. It can be sensitive to noise and outliers, leading to overfitting if the model performs well on this specific subset but poorly on unseen data.
     - Risk of overfitting: With less validation data, it becomes harder to detect overfitting, where the model performs well on the validation set but poorly on unseen data.

The choice of the percentage allocated to the validation set depends on several factors, including the size of your overall dataset, the complexity of your model, and the available computational resources. It's often a trade-off between having a reliable estimate of performance (larger validation set) and ensuring the model is adequately trained (larger training set).

Typically, a common split is 70-80% for training and 20-30% for validation. However, this can vary depending on the specific problem and dataset. You may also consider techniques like cross-validation, which involve splitting the data into multiple subsets for validation to get a more robust estimate of model performance.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


4 ans)Increasing the number of iterations during model training can help in some cases when dealing with a small training dataset or a small validation dataset, but it's important to understand the limitations and potential risks associated with this approach:

Advantages of Increasing Iterations with Small Datasets:

1)Improved Learning: With a small training dataset, increasing the number of iterations can allow the model to learn more from the limited available data. It can help the model better fit the training data and capture more complex patterns.

2)Fine-Tuning: In some cases, a small dataset may not initially provide enough information for the model to converge to an optimal solution. Increasing the iterations can allow the model to continue fine-tuning its parameters, potentially leading to better performance.

Limitations and Risks:

1)Overfitting: Increasing the number of iterations without appropriate regularization can lead to overfitting, especially when working with small datasets. The model may become too complex and start to memorize noise in the training data rather than learning meaningful patterns. This can result in poor generalization to new data.

2)Validation Set Size: If the validation dataset is also small, it may not be able to effectively detect overfitting or provide a reliable estimate of model performance. In such cases, the model's validation performance may not be a good indicator of how it will perform on new, unseen data.

3)Early Stopping: When working with small datasets, it's crucial to monitor the validation performance closely and consider using early stopping to prevent overfitting. Early stopping involves halting the training process when the validation performance starts to degrade.

4)Data Augmentation: In addition to increasing iterations, you can also consider data augmentation techniques to artificially increase the effective size of your training dataset. Data augmentation involves creating new training examples by applying random transformations to your existing data. This can help mitigate overfitting and improve model generalization.

In summary, increasing the number of iterations can be a strategy to improve model performance when dealing with small training or validation datasets. However, it must be done cautiously to avoid overfitting. Regularization techniques, careful monitoring of validation performance, and early stopping should be used in conjunction with increased iterations to strike the right balance between model complexity and generalization, especially in situations where data is limited.


3 ans)In the context of machine learning, the number of iterations typically refers to the number of times a training algorithm updates the model's parameters using the training data (e.g., gradient descent iterations in neural network training or the number of iterations in an iterative optimization algorithm). The number of iterations can have an impact on the estimate of model performance in the following ways:

1)Training Performance: Increasing the number of iterations during model training can lead to better performance on the training data. The model becomes more finely tuned to fit the training data, which may result in lower training error.

2)Overfitting: However, increasing the number of iterations without appropriate regularization can also lead to overfitting. Overfitting occurs when the model becomes too complex and starts to capture noise in the training data, reducing its ability to generalize to new, unseen data.

3)Validation Performance: The number of iterations during training can affect the model's performance on the validation data. Initially, as you increase the number of iterations, the model's validation performance may improve because it is learning better representations of the data. However, if you continue to increase iterations beyond a certain point, you may start to see diminishing returns or even a decrease in validation performance due to overfitting.

4)Early Stopping: To mitigate the risk of overfitting, it's common to use techniques like early stopping. Early stopping involves monitoring the validation performance during training and stopping the training process when the validation performance begins to degrade. This helps find a balance between model complexity and generalization.

5)Cross-Validation: When using cross-validation, the number of iterations can affect the results. With more iterations, you may see higher variance in the cross-validation results because each fold of the cross-validation will have a more finely tuned model. Conversely, with fewer iterations, the results may have lower variance but may not capture the full learning potential of the model.

In summary, the effect of the number of iterations on the estimate of model performance is a trade-off between improved training performance and the risk of overfitting. It's essential to find an appropriate number of iterations that balances these factors. Techniques like early stopping and model evaluation using cross-validation can help you determine the optimal number of iterations for your specific problem, preventing overfitting while achieving good generalization. Ultimately, the goal is to achieve the best performance on unseen data, such as the test set, rather than just optimizing performance on the training or validation data.







2 ans)Averaging the validation accuracy across multiple splits in techniques like k-fold cross-validation provides a more accurate estimate of how well your model is likely to perform on unseen data, including the test set. However, it's important to clarify what "accurate" means in this context.

1)Generalization Estimate: Cross-validation provides a more reliable estimate of how well your model generalizes to unseen data from the same distribution as your training data. It simulates the process of training and evaluating your model on different subsets of the data, reducing the impact of randomness in the initial data split. This estimate is valuable for understanding the model's overall performance and is typically a better indicator of how the model will perform on new, unseen data compared to a single train-validation split.

2)Variance Estimate: Cross-validation also provides insight into the variance or stability of your model's performance. By computing the mean and standard deviation of the validation accuracy across multiple folds, you can get a sense of how consistent the model's performance is. A lower standard deviation indicates that the model's performance is less sensitive to different data splits, which is a valuable aspect of model evaluation.

3)Hyperparameter Tuning: Cross-validation is particularly useful for hyperparameter tuning because it allows you to systematically evaluate different hyperparameter settings and select the ones that consistently perform well across multiple cross-validation runs. This, in turn, can lead to a model that is more likely to perform well on the test set.

4)Model Comparison: If you are comparing multiple models or algorithms, cross-validation helps you make more informed choices by providing a more accurate and consistent measure of their relative performance.

5)While cross-validation provides more accurate and reliable estimates of model performance compared to a single train-validation split, it's important to remember that the accuracy estimates obtained through cross-validation are still estimates. The actual performance on a specific test set can vary, but cross-validation gives you a better understanding of the model's expected performance and its variability.

Ultimately, cross-validation is a valuable tool for assessing and comparing models, selecting hyperparameters, and gaining confidence in your model's ability to generalize to new, unseen data, including the test set.








1 ans)Yes, averaging the validation accuracy across multiple splits in techniques like k-fold cross-validation can indeed give more consistent and reliable results compared to relying on a single validation split. This is one of the main benefits of using cross-validation. Here's why averaging helps:

1)Reduced Variability: Cross-validation helps reduce the impact of randomness in the initial data split. By repeating the training and validation process multiple times with different data partitions, you get a better sense of your model's performance across various subsets of the data. This reduces the chances of getting overly optimistic or pessimistic estimates of model performance due to a single lucky or unlucky split.

2)Robustness to Data Distribution: If your dataset has some inherent variability or heterogeneity, cross-validation can help ensure that the model's performance is consistently evaluated across different parts of the data distribution. It helps capture how well the model generalizes to different subsets of the data.

3)More Informative Results: By averaging the results of multiple cross-validation runs, you obtain more informative statistics, such as mean accuracy and standard deviation. These statistics can provide a better overall picture of your model's performance and its variability.

4)Hyperparameter Tuning: Cross-validation is especially useful when tuning hyperparameters. You can try different hyperparameter settings in each fold and select the ones that consistently perform well across multiple runs. This reduces the risk of overfitting hyperparameters to a specific validation set.

5)Model Selection: If you are comparing multiple models or algorithms, cross-validation allows you to assess their performance more reliably and make informed choices.

However, it's important to note that while cross-validation provides a more robust estimate of model performance, it also requires more computation and time compared to a single train-validation split. The number of folds (k) in cross-validation should be chosen carefully, as a higher k value results in more splits but also increases computational cost. Common choices for k include 5-fold and 10-fold cross-validation, but the optimal value may vary depending on your dataset's size and characteristics.

In summary, averaging the validation accuracy across multiple splits in cross-validation is a valuable technique to obtain more consistent and reliable estimates of your model's performance, making it a widely used practice in machine learning.






